In [2]:

%%pyspark
df = spark.read.load('abfss://housepricing@datalakezain.dfs.core.windows.net/input/housing_in_london_monthly_variables.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(15))

StatementMeta(sparkpool, 5, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, d0104ca8-b33b-4361-b24b-b84ce827b491)

In [27]:
import pyspark.sql.functions as f
from pyspark.sql.types import *


## Adding year column
df=df.withColumn('years',f.year(f.to_timestamp('date', 'yyyy-MM-dd')))


df.printSchema()

## Changing data type of coloums
df=df.withColumn('houses_sold',df['houses_sold'].cast(FloatType()))
df=df.withColumn('average_price',df['average_price'].cast(FloatType()))

df.printSchema()


StatementMeta(sparkpool, 5, 28, Finished, Available)

root
 |-- date: string (nullable = true)
 |-- area: string (nullable = true)
 |-- average_price: string (nullable = true)
 |-- code: string (nullable = true)
 |-- houses_sold: float (nullable = true)
 |-- no_of_crimes: string (nullable = true)
 |-- borough_flag: string (nullable = true)
 |-- years: integer (nullable = true)

root
 |-- date: string (nullable = true)
 |-- area: string (nullable = true)
 |-- average_price: float (nullable = true)
 |-- code: string (nullable = true)
 |-- houses_sold: float (nullable = true)
 |-- no_of_crimes: string (nullable = true)
 |-- borough_flag: string (nullable = true)
 |-- years: integer (nullable = true)



In [30]:
##creating yearly data of houses sold 

yearly_data=df.groupby('years','area').sum('houses_sold')

yearly_data.show(1)

StatementMeta(sparkpool, 5, 31, Finished, Available)

+-----+-----+----------------+
|years| area|sum(houses_sold)|
+-----+-----+----------------+
| 1997|brent|          4249.0|
+-----+-----+----------------+
only showing top 1 row



In [42]:
## checking unique areas
yearly_data.select('area').distinct().count()

StatementMeta(sparkpool, 5, 43, Finished, Available)

45

In [45]:
##creating yearly data of average price

yearly_data2=df.groupby('years','area').avg('average_price')
yearly_data2.count()

StatementMeta(sparkpool, 5, 46, Finished, Available)

1170

In [48]:
## joining 2 dataframes 

yearly_data=yearly_data2.join(yearly_data,['years','area'])

StatementMeta(sparkpool, 5, 49, Finished, Available)

In [50]:
# Populate a temporary view so we can query from SQL
yearly_data.createOrReplaceTempView("yearly_data")


StatementMeta(sparkpool, 5, 51, Finished, Available)

In [57]:
%%pyspark

yearly_data.write.parquet('abfss://housepricing@datalakezain.dfs.core.windows.net/input/yearly_data'+'.parquet') 



StatementMeta(sparkpool, 5, 58, Finished, Available)

In [58]:
a=spark.read.load('abfss://housepricing@datalakezain.dfs.core.windows.net/input/yearly_data.parquet', header=True)

StatementMeta(sparkpool, 5, 59, Finished, Available)

In [59]:
a.show(1)

StatementMeta(sparkpool, 5, 60, Finished, Available)

+-----+-----+------------------+----------------+
|years| area|avg(average_price)|sum(houses_sold)|
+-----+-----+------------------+----------------+
| 1997|brent| 86749.08333333333|          4249.0|
+-----+-----+------------------+----------------+
only showing top 1 row

